In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from helpers import *
from implementations import *

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [13]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0, encodable_th=1, switch_encoding=True)

In [6]:
tX_train

array([[ 1.        ,  0.06833197,  0.40768027, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  0.55250482,  0.54013641, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  3.19515553,  1.09655998, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [ 1.        ,  0.31931645, -0.13086367, ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        , -0.84532397, -0.30297338, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.66533608, -0.25352276, ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
tX_train.shape, tX_test.shape

((250000, 24), (568238, 24))

In [44]:
param_grid = {
    'lambda_': np.logspace(-4, 0, 5),
    'degree': list(range(1, 4)),
    'max_iters': 100,
    'cont_features': [cont_features]
}
metrics, params = logistic_regression_cv(ty_train, tX_train, param_grid=param_grid)

In [46]:
metrics

{'loss': 0.48832368060485054,
 'accuracy': 76.5648,
 'f1_score': 0.6261136027521852}

In [16]:
from implementations import reg_logistic_regression, build_poly
tX_train_poly = build_poly(tX_train, degree=2, cont_features=cont_features)
weights, loss = reg_logistic_regression(ty_train, tX_train_poly, max_iters=1000, lambda_=0.001)

KeyboardInterrupt: 

In [15]:
from helpers import predict_logistic, compute_accuracy, compute_f1
ty_train_pred = predict_logistic(weights, tX_train_poly)
train_accuracy = compute_accuracy(ty_train, ty_train_pred)
train_f1 = compute_f1(ty_train, ty_train_pred)
train_accuracy, train_f1

(77.8396, 0.6456535782586171)

In [50]:
tX_test.shape

(568238, 24)

In [51]:
tX_test_poly = build_poly(tX_test, params['degree'], cont_features)

In [52]:
tX_test_poly.shape

(568238, 44)

In [54]:
from datetime import datetime
method = 'reg_logistic_regression'
time = datetime.now().strftime('%Y%m%dH%H%M%S')
OUTPUT_PATH = f'../submissions/submission_{method}_{time}'
y_pred = predict_logistic(weights, tX_test_poly)
y_pred = replace_values(y_pred, from_val=0, to_val=-1)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)